In [1]:
print('Module 2: Compound collection')

Module 2: Compound collection


In [2]:
#Python3 is required for the chembl api
#It is so sad that I have to jump from python2 to python3 because of this...

In [3]:
import argparse
from chembl_webresource_client.new_client import new_client
import pandas as pd
import numpy as np
#pd.set_option('display.max_columns', None)

# Parse input

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('-i','--input', help = "The target list for compound collection. Or simply the generated file from the Module 1", required=True)
parser.add_argument('-f','--file_name', help = "A name to save out files...", required=True)
args = parser.parse_args()

# Make functions

In [4]:
def uniprot_id_chembl_id(uniprot_id):
    print('Transfering the uniprot id to the chembl id...')

    target = new_client.target
    uniprot_id = uniprot_id
    res = target.filter(target_components__accession=uniprot_id)

    res = pd.DataFrame(res)
    return res

In [5]:
def protein_chembl_id(res):
    i=0
    while i < res.shape[0]:
        if res['target_type'][i] == 'SINGLE PROTEIN':
            target_chembl_id = res['target_chembl_id'][i]
        i=i+1
    return target_chembl_id

In [6]:
def get_compounds(retrieved_id):
    print('Collecting compounds for '+retrieved_id)
    activity = new_client.activity
    res = activity.filter(target_chembl_id=retrieved_id)
    res = pd.DataFrame(res)
    print (str(len(res))+' compounds colllected')
    return res

In [7]:
def analysis_to_collected_cmpds(combined):
    test_type=[]
    number=[]
    assay=pd.DataFrame()
    for i in pd.unique(combined['standard_type']):
        test_type.append(i)
        number.append(combined.loc[combined['standard_type'] == i].shape[0])
        #print (i, combined.loc[combined['standard_type'] == i].shape[0])
    assay['type']=test_type
    assay['count']=number
    assay = assay.sort_values('count', ascending=False, ignore_index=True)
    print(str(assay.shape[0])+' values types')
    print(assay.iloc[0,0] + ' has the most record(s) - ' + str(assay.iloc[0,1]) + ' records')
    return assay

In [8]:
def remove_dup(combined,assay):
    combined = combined.loc[combined['standard_type'] == assay.iloc[0,0]]
    combined_one = combined.drop_duplicates(subset=['molecule_chembl_id'])
    print('After removing duplicates, ' + str(combined_one.shape[0]) + ' compounds remains with ' + assay.iloc[0,0]) 
    return combined_one

In [9]:
def active_cmpds(training_data):
    training_data["standard_value"] = pd.to_numeric(training_data["standard_value"])
    if str(training_data["standard_type"].iloc[0]) == 'Inhibition':
        training_data_active = training_data.loc[training_data['standard_value'] >= 50]
        print (str(training_data_active.shape[0]) + " active compounds collected with standard value > 50%") 
    else:
        training_data_active = training_data.loc[training_data['standard_value'] <= 1000]
        print (str(training_data_active.shape[0]) + " active compounds collected with standard value < 1000 nM") 
    return training_data_active

In [10]:
def inactive_cmpds(training_data):
    training_data["standard_value"] = pd.to_numeric(training_data["standard_value"])
    if str(training_data["standard_type"].iloc[0]) == 'Inhibition':
        training_data_inactive = training_data.loc[training_data['standard_value'] < 50]
        print (str(training_data_inactive.shape[0]) + " inactive compounds collected with standard value < 50%")         
    else:
        training_data_inactive = training_data.loc[training_data['standard_value'] > 1000]
        print (str(training_data_inactive.shape[0]) + " inactive compounds collected with standard value > 1000 nM") 
    return training_data_inactive

In [11]:
def write_out(table,name):
    file_name=str(name)
    table.to_csv(file_name+'.csv',index=False)
    print ("Collected info is written to the disk.")
    return

# Use functions

In [ ]:
df=pd.read_csv(args.input)
combined=pd.DataFrame()
print (args.input + ' is recognized')

for uniprot_id_one in df['uniprot id']:
    res=uniprot_id_chembl_id(uniprot_id_one)
    if np.array(res) != []:
        chembl_id = protein_chembl_id(res)
        table=get_compounds(chembl_id)
        combined = pd.concat([combined,table],ignore_index=True)
    
write_out(combined,args.file_name)

In [ ]:
assay=analysis_to_collected_cmpds(combined)
write_out(assay,'assay_info_' + args.file_name)

In [ ]:
training_data=remove_dup(combined,assay)
actives=active_cmpds(training_data)
write_out(actives,'actives_' + args.file_name)
inactives=inactive_cmpds(training_data)
write_out(inactives,'inactives_' + args.file_name)

# Example

In [12]:
df=pd.read_csv('P48729.csv')
combined=pd.DataFrame()

In [ ]:

for uniprot_id_one in df['uniprot id']:
    print ('Uniprot id: ' +uniprot_id_one)
    res=uniprot_id_chembl_id(uniprot_id_one)
    if np.array(res) != []:
        chembl_id = protein_chembl_id(res)
        table=get_compounds(chembl_id)
        combined = pd.concat([combined,table],ignore_index=True)
    
write_out(combined,'collected_for_CSNK1A')

Uniprot id: P48729
Transfering the uniprot id to the chembl id...


/Applications/anaconda/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  after removing the cwd from sys.path.


2563 compounds colllected
Uniprot id: P67962
Transfering the uniprot id to the chembl id...
Uniprot id: P67827
Transfering the uniprot id to the chembl id...


/Applications/anaconda/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  after removing the cwd from sys.path.


11 compounds colllected
Uniprot id: Q8N752
Transfering the uniprot id to the chembl id...
287 compounds colllected
Uniprot id: P49674
Transfering the uniprot id to the chembl id...
1279 compounds colllected
Uniprot id: P35508
Transfering the uniprot id to the chembl id...
Uniprot id: Q9Y6M4
Transfering the uniprot id to the chembl id...
2403 compounds colllected
Uniprot id: Q9HCP0
Transfering the uniprot id to the chembl id...


In [156]:
assay=analysis_to_collected_cmpds(combined)
write_out(assay,'assay_info'+'_collected_for_CDK2')

35 values types
IC50 has the most record(s) - 8315 records
Collected compounds are written to the disk.


In [155]:
training_data=remove_dup(combined,assay)
actives=active_cmpds(training_data)
write_out(actives,'actives_collected_for_CDK2')
inactives=inactive_cmpds(training_data)
write_out(inactives,'inactives_collected_for_CDK2')

After removing duplicates, 5128 compounds remains with IC50
1913 active compounds collected with standard value < 100 nM
Collected compounds are written to the disk.
3108 inactive compounds collected with standard value > 100 nM
Collected compounds are written to the disk.


In [12]:
combined = pd.read_csv('P08684_compounds_collection.csv')

/Applications/anaconda/envs/python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
assay = pd.read_csv('assay_info_P08684_compounds_collection.csv')

In [16]:
training_data=remove_dup(combined,assay)
actives=active_cmpds(training_data)
write_out(actives,'actives_collected_for_CYP3A4')
inactives=inactive_cmpds(training_data)
write_out(inactives,'inactives_collected_for_CYP3A$')

After removing duplicates, 16572 compounds remains with AC50


KeyError: 0

In [20]:
    training_data["standard_value"] = pd.to_numeric(training_data["standard_value"])



In [21]:
training_data_active = training_data.loc[training_data['standard_value'] <= 1000]

In [33]:
training_data["standard_type"][0]

KeyError: 0

In [23]:
str(training_data["standard_type"][0])

KeyError: 0

In [19]:
    training_data["standard_value"] = pd.to_numeric(training_data["standard_value"])
    if str(training_data["standard_type"][0]) == 'Inhibition':
        training_data_active = training_data.loc[training_data['standard_value'] >= 50]
        print (str(training_data_active.shape[0]) + " active compounds collected with standard value > 50%") 
    else:
        training_data_active = training_data.loc[training_data['standard_value'] <= 1000]
        print (str(training_data_active.shape[0]) + " active compounds collected with standard value < 1000 nM")

KeyError: 0